In [2]:
import nltk
import json
import pickle
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers.legacy import SGD

In [3]:
# get root word of any words Dibby doesn't know
lemmatizer = WordNetLemmatizer()

# load file of intents for processing
intents = json.loads(open("intent.json").read())
    
# creating empty lists to store contents of intents ****
# words contains all words in the patterns lists
words = [] 
# classes contans all tags
classes = []
# documents contains the patterns and their associated tags
documents = [] 

# ignore punctuation
ignore_letter = ["?", "!", ".", ","]

In [4]:
# iterate through all intents
for intent in intents['intents']:
        # iterate through all patterns for a single intent
    for pattern in intent['patterns']:
        # word_list contains all words in the "patterns" list
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # associate the patterns w/ their respective tags
        documents.append((word_list, intent['tag']))
        # append tags to the class list if not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# modify words list so it only contans the root words
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letter]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
# save words & classes list to bnary files
pickle.dump(words, open('words.pk1', 'wb'))
pickle.dump(classes, open('classes.pk1', 'wb'))

In [7]:
# neural networks work w/ numerical values, not strings
    # classify data (words) as 0's and 1's
# training will store data (patterns) used for training
training = []
# output will store data (classes) used for training
output = []
# output_empty will store a 0 for every class in intent.json
output_empty = [0]*len(classes)

In [8]:
for document in documents:
    # bag contans 0 if a word in words isn't in the pattern, 1 if it is
    bag = []
    # list of words associated w/ certain tag
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    # output row indicates which class is associated with the current "bag" (patterns kinda)
    output_row[classes.index(document[1])] = 1
    training.append(bag)
    output.append(output_row)

In [9]:
# make data a numpy array
training = np.array(training)
output = np.array(output)

train_x = list(training)
train_y = list(output)

In [10]:
# creating a Sequential machine learning model 
model = Sequential() 
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(train_y[0]), activation='softmax')) 

In [11]:
# compiling the model 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy', 
			optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(train_x), np.array(train_y), 
				epochs=200, batch_size=5, verbose=1) 

Epoch 1/200


/Users/nisha/miniforge3/lib/python3.10/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


16/16 [==============================] - 0s 664us/step - loss: 2.2762 - accuracy: 0.1266
Epoch 2/200
16/16 [==============================] - 0s 519us/step - loss: 2.1722 - accuracy: 0.2658
Epoch 3/200
16/16 [==============================] - 0s 513us/step - loss: 1.9746 - accuracy: 0.2785
Epoch 4/200
16/16 [==============================] - 0s 571us/step - loss: 1.7611 - accuracy: 0.4430
Epoch 5/200
16/16 [==============================] - 0s 530us/step - loss: 1.5537 - accuracy: 0.5316
Epoch 6/200
16/16 [==============================] - 0s 522us/step - loss: 1.3893 - accuracy: 0.5823
Epoch 7/200
16/16 [==============================] - 0s 557us/step - loss: 1.1774 - accuracy: 0.5949
Epoch 8/200
16/16 [==============================] - 0s 500us/step - loss: 1.0994 - accuracy: 0.6582
Epoch 9/200
16/16 [==============================] - 0s 638us/step - loss: 0.9315 - accuracy: 0.7342
Epoch 10/200
16/16 [==============================] - 0s 512us/step - loss: 0.8601 - accuracy: 0.7215
E

In [12]:
# saving the model 
model.save('my_model.keras')